In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
delays = pd.read_csv('csv_originals/delays.csv', encoding = "ISO-8859-1")

In [3]:
delays.head()

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle
0,1/1/2014,02:06,Wednesday,HIGH PARK STATION,SUDP,3,7,W,BD,5001
1,1/1/2014,02:40,Wednesday,SHEPPARD STATION,MUNCA,0,0,NaN,YU,0
2,1/1/2014,03:10,Wednesday,LANSDOWNE STATION,SUDP,3,8,W,BD,5116
3,1/1/2014,03:20,Wednesday,BLOOR STATION,MUSAN,5,10,S,YU,5386
4,1/1/2014,03:29,Wednesday,DUFFERIN STATION,MUPAA,0,0,E,BD,5174


In [4]:
delays.isnull().any()
## Looks like Code, Bound and Line coutain NaNs

Date         False
Time         False
Day          False
Station      False
Code          True
Min Delay    False
Min Gap      False
Bound         True
Line          True
Vehicle      False
dtype: bool

## Date

In [5]:
dates = pd.to_datetime(delays['Date'])

In [6]:
years = dates.dt.year
months = dates.dt.month
days = dates.dt.day
years.unique(), months.unique(), days.unique()

(array([2014, 2015, 2016, 2017]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]))

The years are between the expected 2014-2017 and the months/days look reasonable so no clean up on *Date*

## Time

In [7]:
hours = pd.to_datetime(delays['Time'], format='%H:%M').dt.hour
minutes = pd.to_datetime(delays['Time'], format='%H:%M').dt.minute
hours.min(), hours.max(), minutes.min(), minutes.max()

(0, 23, 0, 59)

Looks like the hours are within 0 - 23 and the minutes are within 0-59 so no clean up needed for *Time*

## Day

In [8]:
delays['Day'].unique()

array(['Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'Monday',
       'Tuesday'], dtype=object)

Days look reasonable so no clean up

Make sure the days from the day column match the day from the date?

In [9]:
dates_day = delays.copy()[['Day', 'Date']]
dates_day['Date'] = pd.to_datetime(dates_day['Date'])
dates_day['Date Day'] = dates_day['Date'].dt.weekday_name
dates_day[dates_day['Date Day'] != dates_day['Day']]

,Day,Date,Date Day


Looks like the *Day* column matches the expected date so no clean up here

## Station

In [10]:
# Source https://www1.toronto.ca/wps/portal/contentonly?vgnextoid=c077c316f16e8410VgnVCM10000071d60f89RCRD&vgnextchannel=7807e03bb8d1e310VgnVCM10000071d60f89RCRD

ttc_stations = pd.read_csv('stations.csv')
ttc_stations.head()

,Rank,Subway/RT Station,To Trains,From Trains,Totals,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1,"Bloor (line 1, Y-U)","103,975","112,214","216,189",NaN,NaN,NaN,NaN,NaN,NaN
1,2,"Yonge (line 2, B-D)","95,407","87,831","183,238",NaN,NaN,NaN,NaN,NaN,NaN
2,3,"St. George (line 1, Y-U)","63,367","72,836","136,203",NaN,NaN,NaN,NaN,NaN,NaN
3,4,"St. George (line 2, B-D)","64,125","64,851","128,976",NaN,NaN,NaN,NaN,NaN,NaN
4,5,Union,"58,259","60,187","118,446",NaN,NaN,NaN,NaN,NaN,NaN


The station in the above data all have their line in brackets but this will cause issues comparing to the delay data.

Strip the station name from the bracket, the delay data has the exact line so we can use this to differentiate the data

In [11]:
ttc_stations['Short Name'] = ttc_stations['Subway/RT Station'].dropna().str.split('(').str.get(0).str.upper().str.strip()

# Fix up SHEPPARD and MAIN to match the delay data better
ttc_stations[ttc_stations['Short Name'] == 'SHEPPARD-YONGE'] = 'SHEPPARD'
ttc_stations[ttc_stations['Short Name'] == 'MAIN STREET'] = 'MAIN'

# Drop any NaNs and grab just the uniques
ttc_stations_names = ttc_stations['Short Name'].dropna().unique()

'''
Some stations have a 'West' version and ideally we want these first since we don't want to
misclasify St. Clear West as St Clair so we reverse sort the names so the longer name is first
'''
ttc_stations_names = np.sort(ttc_stations_names)[::-1]

ttc_stations_names

array(['YORKDALE', 'YORK MILLS', 'YONGE', 'WOODBINE', 'WILSON',
       'WELLESLEY', 'WARDEN', 'VICTORIA PARK', 'UNION', 'SUMMERHILL',
       'ST. PATRICK', 'ST. GEORGE', 'ST. CLAIR WEST', 'ST. CLAIR',
       'ST. ANDREW', 'SPADINA', 'SHERBOURNE', 'SHEPPARD',
       'SCARBOROUGH CENTRE', 'RUNNYMEDE', 'ROYAL YORK', 'ROSEDALE',
       "QUEEN'S PARK", 'QUEEN', 'PAPE', 'OSSINGTON', 'OSGOODE', 'OLD MILL',
       'NORTH YORK CENTRE', 'MUSEUM', 'MIDLAND', 'MCCOWAN', 'MAIN',
       'LESLIE', 'LAWRENCE WEST', 'LAWRENCE EAST', 'LAWRENCE', 'LANSDOWNE',
       'KIPLING', 'KING', 'KENNEDY', 'KEELE', 'JANE', 'ISLINGTON',
       'HIGH PARK', 'GREENWOOD', 'GLENCAIRN', 'FINCH', 'ELLESMERE',
       'EGLINTON WEST', 'EGLINTON', 'DUPONT', 'DUNDAS WEST', 'DUNDAS',
       'DUFFERIN', 'DOWNSVIEW', 'DONLANDS', 'DON MILLS', 'DAVISVILLE',
       'COXWELL', 'COLLEGE', 'CHRISTIE', 'CHESTER', 'CASTLE FRANK',
       'BROADVIEW', 'BLOOR', 'BESSARION', 'BAYVIEW', 'BAY', 'BATHURST'], dtype=object)

Save the original stations and count how many there were

In [12]:
delays['Station_original'] = delays['Station']
stations = delays['Station']
# 76801 Way too many stations!
stations.count()

76801

Fixing up some inconsistencies with the delays stations. The St are need to be normalized and some common typos fixed

In [13]:
def fix_station(station):
    if station.startswith('ST'):
#         return station.replace('ST.', 'ST. ').replace('ST ', 'ST. ')
        return station.replace('ST.', 'ST. ').replace('ST ', 'ST. ').replace('ST.  ', 'ST. ')
    elif station == 'NORTH YORK CTR STATION' or station == 'NORTH YORK CENTER' or station == 'NORTH YORK CENTER STAT':
        return 'NORTH YORK CENTRE'
    elif '0SSINGTON' in station:
        return station.replace('0SSINGTON', 'OSSINGTON')
    elif 'BESSARIAN' in station or 'BESSARRION' in station:
        return 'BESSARION'
    elif 'BUTHURST' in station:
        return 'BATHURST'
    elif 'SCARB' in station or 'SCARBOROUGH' in station or 'SCAB' in station or 'SCAR' in station and 'RAPID' not in station:
        return 'SCARBOROUGH CENTRE'
    elif 'DOWNVIEW' in station:
        return 'DOWNSVIEW'
    else: 
        return station
    
# Store the newly fixed stations in it's own column  
delays['Station_Fixed'] = delays['Station_original'].apply(fix_station)
delays

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,Station_original,Station_Fixed
0,1/1/2014,02:06,Wednesday,HIGH PARK STATION,SUDP,3,7,W,BD,5001,HIGH PARK STATION,HIGH PARK STATION
1,1/1/2014,02:40,Wednesday,SHEPPARD STATION,MUNCA,0,0,NaN,YU,0,SHEPPARD STATION,SHEPPARD STATION
2,1/1/2014,03:10,Wednesday,LANSDOWNE STATION,SUDP,3,8,W,BD,5116,LANSDOWNE STATION,LANSDOWNE STATION
3,1/1/2014,03:20,Wednesday,BLOOR STATION,MUSAN,5,10,S,YU,5386,BLOOR STATION,BLOOR STATION
4,1/1/2014,03:29,Wednesday,DUFFERIN STATION,MUPAA,0,0,E,BD,5174,DUFFERIN STATION,DUFFERIN STATION
5,1/1/2014,07:31,Wednesday,NORTH YORK CENTRE STAT,MUNCA,0,0,NaN,YU,0,NORTH YORK CENTRE STAT,NORTH YORK CENTRE STAT
6,1/1/2014,07:32,Wednesday,SHEPPARD STATION,MUNCA,0,0,NaN,YU,0,SHEPPARD STATION,SHEPPARD STATION
7,1/1/2014,07:34,Wednesday,RUNNYMEDE STATION,MUNCA,0,0,NaN,BD,0,RUNNYMEDE STATION,RUNNYMEDE STATION
8,1/1/2014,07:34,Wednesday,QUEEN STATION EAST SID,MUNCA,0,0,NaN,YU,0,QUEEN STATION EAST SID,QUEEN STATION EAST SID
9,1/1/2014,07:53,Wednesday,ST ANDREW STATION,MUNCA,0,0,NaN,YU,0,ST ANDREW STATION,ST. ANDREW STATION


Now that we have a list of known stations and a cleaner list of stations lets take our best guess at the 'Normalized Station'

**Warning** sometimes the above code is slow or gets stuck you can kill the kernal and restart

In [14]:
def estimate_station(original_station):
    for station_name in ttc_stations_names:        
        if station_name in original_station:
            return station_name
        
    return np.NaN
delays['Station'] = delays['Station_Fixed'].apply(estimate_station)
delays.head()

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,Station_original,Station_Fixed
0,1/1/2014,02:06,Wednesday,HIGH PARK,SUDP,3,7,W,BD,5001,HIGH PARK STATION,HIGH PARK STATION
1,1/1/2014,02:40,Wednesday,SHEPPARD,MUNCA,0,0,NaN,YU,0,SHEPPARD STATION,SHEPPARD STATION
2,1/1/2014,03:10,Wednesday,LANSDOWNE,SUDP,3,8,W,BD,5116,LANSDOWNE STATION,LANSDOWNE STATION
3,1/1/2014,03:20,Wednesday,BLOOR,MUSAN,5,10,S,YU,5386,BLOOR STATION,BLOOR STATION
4,1/1/2014,03:29,Wednesday,DUFFERIN,MUPAA,0,0,E,BD,5174,DUFFERIN STATION,DUFFERIN STATION


Next lets check out the stations that are still unknown -- also add a check here removing all the stations where the Line is null since those data points likely have issues with the stations

In [15]:
# delays
unknown_station = delays[delays['Station'].isnull() & (delays['Line'].notnull())]
len(unknown_station['Station_Fixed'].unique())
# unknown_station['Station'].unique()
print(len(unknown_station))
unknown_station.groupby('Station_Fixed').size().sort_values(ascending=False)

342


Station_Fixed
TORONTO TRANSIT COMMIS    186
SRT LINE                   20
DANFORTH DIVISION          10
TRANSIT CONTROL CENTRE      6
MCBRIEN BUILDING            4
CANADIAN ARMED FORCES       3
VARIOUS                     3
CHANGE OVERS / GENERAL      3
SYSTEM WIDE                 2
GENERAL DELAY / CHANGE      2
TRANSIT CONTROL             2
YUS                         2
YOUNGE/UNIVERSITY - LI      2
GREEWOOD STATION            2
APPROACHING VICTORIA P      2
YOUNG UNIVERSITY SUBWA      2
FENMAR AND WESTON           1
DUPLEX AVENUE               1
DOWNSIVEW STATION (APP      1
DOWSVIEW STATION            1
EGLINGTON STATION           1
EXITING ST CLAIR WEST       1
YUS/BD/SRT                  1
FULL LINE                   1
GLENACAIRN STATION          1
DOWNSIVEW STATION           1
GLENCARIAN (APPROACHIN      1
GLENCARIAN STATION          1
GREEENWOOD WYE (YARD A      1
GREEN                       1
                         ... 
YOUNGE/UNIVERSITY-BLOO      1
YOUNGE/UNIVERSITY/SAPD    

It looks like we weren't able to classify 342 of the original stations but from a quick glance some of of these aren't real station (like TRANISIT CONTORL, DANFORTH DIVISION )

Interestingly some of there seem like entire lines or Systems (SYSTEM WIDE, SRT LINE ) which may be worth looking at individually than the stations

In [16]:
# Quick glance at the final stations
delays.groupby('Station').size()

Station
BATHURST               684
BAY                    542
BAYVIEW                316
BESSARION              187
BLOOR                 3513
BROADVIEW              774
CASTLE FRANK           443
CHESTER                628
CHRISTIE               888
COLLEGE                698
COXWELL               1228
DAVISVILLE            1347
DON MILLS              726
DONLANDS               752
DOWNSVIEW             1970
DUFFERIN               497
DUNDAS                 807
DUNDAS WEST            915
DUPONT                 757
EGLINTON              1853
EGLINTON WEST         1013
ELLESMERE              193
FINCH                 2418
GLENCAIRN              608
GREENWOOD             1356
HIGH PARK              475
ISLINGTON             1808
JANE                  1482
KEELE                 2139
KENNEDY               4105
                      ... 
MUSEUM                 483
NORTH YORK CENTRE      566
OLD MILL               912
OSGOODE                401
OSSINGTON              822
PAPE                

## Line

In [17]:
# Similar to station store the original lines
delays['Line_ori'] = delays['Line']

Based on the metadata that came with the data we know there are only 4 real lines so we can map these with their full name which will also convert the rest to null

In [18]:
expected_lines = {'BD': 'Bloor-Danforth', 'YU': 'Yonge-University', 'SHP' : 'Sheppard', 'SRT' : 'Scarborough RT'}
delays['Line'] = delays['Line_ori'].map(expected_lines)

## Bound

The meta data says that Bound should be a direction N/S/E/W so we can likely drop the B/R/Y which may have been typos

In [19]:
delays.groupby('Bound').size()

Bound
B       35
E    15281
N    12882
R        1
S    15271
W    16876
Y        2
dtype: int64

In [20]:
expected_directions = {'E': 'East', 'N': 'North', 'W':'West', 'S':'South'}
delays['Bound'] = delays['Bound'].map(expected_directions)
delays.groupby('Bound').size()

Bound
East     15281
North    12882
South    15271
West     16876
dtype: int64

## Vehicle

Empty vehicle numbers seem to be treated as 0 but NaN or None is likely more appropriate here

In [21]:
delays['Vehicle'] = delays['Vehicle'].replace(0, np.NaN)

## Codes

We recieved the full code names from the data set so we can put the description in the table

In [22]:
codes = pd.read_csv('codes.csv')
# Likely due to an encoding issue but the Code column is 'SUB RMENU CODE' so rename it to Code
codes['Code'] = codes['SUB RMENU CODE']
codes['Code Description'] = codes['CODE DESCRIPTION']

To get the codes into our delays dataframe we need to merge the codes

In [23]:
delays = delays.merge(codes, how='left', on='Code')
delays

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,Station_original,Station_Fixed,Line_ori,SUB RMENU CODE,CODE DESCRIPTION,Code Description
0,1/1/2014,02:06,Wednesday,HIGH PARK,SUDP,3,7,West,Bloor-Danforth,5001.0,HIGH PARK STATION,HIGH PARK STATION,BD,SUDP,Disorderly Patron,Disorderly Patron
1,1/1/2014,02:40,Wednesday,SHEPPARD,MUNCA,0,0,NaN,Yonge-University,NaN,SHEPPARD STATION,SHEPPARD STATION,YU,NaN,NaN,NaN
2,1/1/2014,03:10,Wednesday,LANSDOWNE,SUDP,3,8,West,Bloor-Danforth,5116.0,LANSDOWNE STATION,LANSDOWNE STATION,BD,SUDP,Disorderly Patron,Disorderly Patron
3,1/1/2014,03:20,Wednesday,BLOOR,MUSAN,5,10,South,Yonge-University,5386.0,BLOOR STATION,BLOOR STATION,YU,MUSAN,Unsanitary Vehicle,Unsanitary Vehicle
4,1/1/2014,03:29,Wednesday,DUFFERIN,MUPAA,0,0,East,Bloor-Danforth,5174.0,DUFFERIN STATION,DUFFERIN STATION,BD,MUPAA,Passenger Assistance Alarm Activated - No Trou...,Passenger Assistance Alarm Activated - No Trou...
5,1/1/2014,07:31,Wednesday,NORTH YORK CENTRE,MUNCA,0,0,NaN,Yonge-University,NaN,NORTH YORK CENTRE STAT,NORTH YORK CENTRE STAT,YU,NaN,NaN,NaN
6,1/1/2014,07:32,Wednesday,SHEPPARD,MUNCA,0,0,NaN,Yonge-University,NaN,SHEPPARD STATION,SHEPPARD STATION,YU,NaN,NaN,NaN
7,1/1/2014,07:34,Wednesday,RUNNYMEDE,MUNCA,0,0,NaN,Bloor-Danforth,NaN,RUNNYMEDE STATION,RUNNYMEDE STATION,BD,NaN,NaN,NaN
8,1/1/2014,07:34,Wednesday,QUEEN,MUNCA,0,0,NaN,Yonge-University,NaN,QUEEN STATION EAST SID,QUEEN STATION EAST SID,YU,NaN,NaN,NaN
9,1/1/2014,07:53,Wednesday,ST. ANDREW,MUNCA,0,0,NaN,Yonge-University,NaN,ST ANDREW STATION,ST. ANDREW STATION,YU,NaN,NaN,NaN


Let see if code codes didn't get translated 

In [24]:
# lets see if any codes don't have description
no_description = delays[delays['Code Description'].isnull()]
print('Number of non decoded values: ', len(no_description))

# Reverse sort by the number of entries of each of this code
no_description.groupby('Code').size().sort_values(ascending=False)

Number of non decoded values:  1778


Code
MUNCA     1561
PUTDN       97
TRNCA       48
XXXXX       27
TUNCA       17
MFO          5
SFDP         4
XXXXXX       3
MTUI         3
MUCSA        2
SFO          2
ETCA         1
MTO          1
PUTN         1
PUSIC        1
SFAE         1
SFAP         1
TUTD         1
ERTO         1
dtype: int64

Looks like most of these are one off's with the exception of *MUNCA* which has 1561 values!
One guess is this was meant to be MUNOA,No Operator Immediately Available - Not E.S.A. Related  or it could be missing from the code list

In [25]:
# List out the codes with the most common one first
delays.groupby('Code Description').size().sort_values(ascending=False)

Code Description
Miscellaneous Speed Control                                   13741
Operator Overspeeding                                         10192
Passenger Assistance Alarm Activated - No Trouble Found        4943
Miscellaneous General Delays                                   4094
Injured or ill Customer (In Station) - Transported             4021
Miscellaneous Other                                            3535
Disorderly Patron                                              2681
Injured or ill Customer (On Train) - Medical Aid Refused       2409
Injured or ill Customer (On Train) - Transported               2093
Operator Violated Signal                                       1766
Escalator/Elevator Incident                                    1721
Door Problems - Faulty Equipment                               1448
Passenger Other                                                1125
Brakes                                                         1095
Misc. Transportation Other - Em

In [26]:
# Not really clean up I just was curcious how many of these were 'Passenger' related
filled = delays['Code Description'].fillna('')
pass_related = filled[filled.str.contains('Passenger')].unique()

In [27]:
pass_delays = delays[delays['Code Description'].isin(pass_related)]
print(len(pass_delays))
pass_delays.groupby('Code Description').size()

6414


Code Description
Door Problems - Passenger Related                           346
Passenger Assistance Alarm Activated - No Trouble Found    4943
Passenger Other                                            1125
dtype: int64

In [30]:
# Remove some not really useful columns before exporting (CODE DESCRIPTION is a duplicate)
delays = delays.drop(['Station_Fixed', 'SUB RMENU CODE', 'CODE DESCRIPTION'], axis=1)

In [31]:
delays.to_csv('ttc_delays_cleaned.csv')

In [33]:
cleaned = pd.read_csv('ttc_delays_cleaned.csv')

In [34]:
cleaned

,Unnamed: 0,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle,Station_original,Line_ori,Code Description
0,0,1/1/2014,02:06,Wednesday,HIGH PARK,SUDP,3,7,West,Bloor-Danforth,5001.0,HIGH PARK STATION,BD,Disorderly Patron
1,1,1/1/2014,02:40,Wednesday,SHEPPARD,MUNCA,0,0,NaN,Yonge-University,NaN,SHEPPARD STATION,YU,NaN
2,2,1/1/2014,03:10,Wednesday,LANSDOWNE,SUDP,3,8,West,Bloor-Danforth,5116.0,LANSDOWNE STATION,BD,Disorderly Patron
3,3,1/1/2014,03:20,Wednesday,BLOOR,MUSAN,5,10,South,Yonge-University,5386.0,BLOOR STATION,YU,Unsanitary Vehicle
4,4,1/1/2014,03:29,Wednesday,DUFFERIN,MUPAA,0,0,East,Bloor-Danforth,5174.0,DUFFERIN STATION,BD,Passenger Assistance Alarm Activated - No Trou...
5,5,1/1/2014,07:31,Wednesday,NORTH YORK CENTRE,MUNCA,0,0,NaN,Yonge-University,NaN,NORTH YORK CENTRE STAT,YU,NaN
6,6,1/1/2014,07:32,Wednesday,SHEPPARD,MUNCA,0,0,NaN,Yonge-University,NaN,SHEPPARD STATION,YU,NaN
7,7,1/1/2014,07:34,Wednesday,RUNNYMEDE,MUNCA,0,0,NaN,Bloor-Danforth,NaN,RUNNYMEDE STATION,BD,NaN
8,8,1/1/2014,07:34,Wednesday,QUEEN,MUNCA,0,0,NaN,Yonge-University,NaN,QUEEN STATION EAST SID,YU,NaN
9,9,1/1/2014,07:53,Wednesday,ST. ANDREW,MUNCA,0,0,NaN,Yonge-University,NaN,ST ANDREW STATION,YU,NaN
